In [1]:
import os
import openai

In [2]:
#os.environ['OPENAI_API_KEY'] = ''
openai.api_key = os.getenv('OPENAI_API_KEY')


In [3]:
prompt = "Give me details about the technology startup called Mimi and Pimo"

In [4]:
response = openai.Completion.create(engine = 'text-davinci-003',
                                    prompt = prompt,
                                    max_tokens=256,
                                    temperature = 0.7)

In [5]:
print(response['choices'][0]['text'])



Mimi and Pimo is an AI-driven technology startup that helps people with their everyday tasks. Founded in 2017, the company has developed a voice-activated assistant that can be used to shop, book travel, and provide information. The company is based in London, England and is funded by venture capital firms including Y Combinator and Octopus Ventures. The company has developed a suite of products including an app, web platform, and a range of devices that give users access to the assistant. The company has also developed an AI-driven chatbot to provide customer service and answer questions. The company has been featured in publications such as TechCrunch and The Guardian.


Model hallucinates, one of the ways of fixing it:

In [6]:
prompt = "Give me details about the technology startup called Mimi and Pimo. Only answer if you are 100% sure that this company exists, otherwise specify, 'I don't know '"

In [7]:
response = openai.Completion.create(engine = 'text-davinci-003',
                                    prompt = prompt,
                                    max_tokens=256,
                                    temperature = 0.7)

In [8]:
print(response['choices'][0]['text'])



I don't know.


## Creating our test prompt

In [9]:
def create_test_prompt(topic,num_questions,num_possible_answers):
    prompt = f"Create a multiple choice quiz on the topic of {topic} consisting of {num_questions} questions."\
            +f" Each question should have {num_possible_answers} options."\
            +f" Also include the correct answer for each question using the starting string 'Correct answer:' "
    return prompt

In [10]:
create_test_prompt("Python",4,4)

"Create a multiple choice quiz on the topic of Python consisting of 4 questions. Each question should have 4 options. Also include the correct answer for each question using the starting string 'Correct answer:' "

In [11]:
response = openai.Completion.create(engine = 'text-davinci-003',
                                    prompt = create_test_prompt('Spotify',4,4),
                                    max_tokens=256,
                                    temperature = 0.7)

In [12]:
print(response['choices'][0]['text'])



1. What is the name of the music streaming service owned by Spotify?
a. iTunes
b. Google Music
c. Amazon Music
d. Apple Music
Correct answer: d. Apple Music

2. How much does Spotify's Premium subscription cost per month?
a. $5
b. $10
c. $15
d. $20
Correct answer: c. $15

3. How many songs does the average user store in their library?
a. 10
b. 50
c. 100
d. 200
Correct answer: b. 50

4. What feature allows you to save your favorite songs for easy access?
a. Playlist
b. Albums
c. Radio
d. Library
Correct answer: d. Library


## Q&A Extraction


In [13]:
def create_student_view(test,num_question):
    student_view = {1:''}
    question_number = 1
    for line in test.split("\n"):
        if not line.startswith("Correct answer:"):
            student_view[question_number]+= line+'\n'
        else:
            if question_number <num_question:
                question_number += 1
                student_view[question_number] = ''
    return student_view

In [14]:
result = create_student_view(response['choices'][0]['text'],4)

In [15]:
for key in result:
    print(result[key])



1. What is the name of the music streaming service owned by Spotify?
a. iTunes
b. Google Music
c. Amazon Music
d. Apple Music


2. How much does Spotify's Premium subscription cost per month?
a. $5
b. $10
c. $15
d. $20


3. How many songs does the average user store in their library?
a. 10
b. 50
c. 100
d. 200


4. What feature allows you to save your favorite songs for easy access?
a. Playlist
b. Albums
c. Radio
d. Library



In [16]:
def extract_answer(test,num_question):
    answers = {1:''}
    question_number = 1
    for line in test.split("\n"):
        if line.startswith("Correct answer:"):
            answers[question_number]+= line+'\n'
            if question_number <num_question:
                question_number += 1
                answers[question_number] = ''
    return answers

In [17]:
extract_answer(response['choices'][0]['text'],4)

{1: 'Correct answer: d. Apple Music\n',
 2: 'Correct answer: c. $15\n',
 3: 'Correct answer: b. 50\n',
 4: 'Correct answer: d. Library\n'}

In [18]:
student_view = create_student_view(response['choices'][0]['text'],4)

In [19]:
answers = extract_answer(response['choices'][0]['text'],4)